In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("..")
import dataloader as dl
import numpy as np
import pandas as pd

from dataloader import Dataset
from baseline_slim_sqs import cast_to_pattern, mine
import bayesian_network
from pattern import Pattern, EmptyPattern
from cover import Cover
import collections
import bidict

In [ ]:
df = dl.load_socbed_bi()
dataset = Dataset(df.copy())
t_patterns = mine(df, True)

t_patterns = list(map(lambda x: cast_to_pattern(list(x), dataset), t_patterns))
t_patterns = list(filter(lambda x: x != None, t_patterns))


# print(t_patterns)
base_bn = bayesian_network.BayesianNetwork()
all_empty_windows = EmptyPattern().find_windows(dataset)
base_bn.learn(dataset, all_empty_windows)
c = Cover(t_patterns, base_bn, dataset)
c.greedy_search() #call once implemented
cover_map = c.get_cover_map()
c.fit_bayesian_networks(t_patterns)


In [2]:
import sys
sys.path.append('..')
import numpy as np
import pandas as pd
import model
import pickle
import collections
import bidict
from pattern import Pattern, EmptyPattern
model_path = '/home/aschoen/my_storage/aschoen/models/joschua_model/CIDDS_xp4-our.pkl'
m = pickle.load(open(model_path,'rb'))
c = m.cover

In [3]:
cnt = collections.Counter(c.get_cover_map().ravel()) # np.unique does not work
used_windows = cnt.keys()

In [4]:
wind_ids = bidict.bidict(enumerate(used_windows))
pattern_ids = bidict.bidict(enumerate(c.pattern_set))

In [5]:
usage_dict = {}
stats = c.get_cover_stats()
usage = stats.get_pattern_usage()
for pattern, u in usage.items():
    if not isinstance(pattern, EmptyPattern):
        usage_dict[pattern_ids.inverse[pattern]] = u

In [ ]:
pattern_file = open("notebooks/data/patterns_with_ids_xp2.txt", "w+")

for i, pattern in pattern_ids.items():
    try:
        u = usage_dict[i]
    except KeyError:
        u = 0
    pattern_file.write(f"+++++ Pattern {i} - Usage {u}  +++++\n")
    pattern_file.write(pattern.get_real_value_repr(dataset))
    pattern_file.write("\n\n")

pattern_file.close()


In [6]:
#replace windows in cover_map with their index
cover_map_ids = np.vectorize(lambda x: wind_ids.inverse[x])(c.get_cover_map())
max_ids = np.max(cover_map_ids, axis=1)
used_pattern = np.vectorize(lambda x: wind_ids[x].pattern if wind_ids[x] != None else None)(max_ids)
used_pattern = np.vectorize(lambda x: pattern_ids.inverse[x] if x != None else -1)(used_pattern)

In [7]:
df = pd.read_csv("/home/aschoen/my_storage/aschoen/dataset/flow_chronicle_dataset/xp5/CIDDS_xp5_train.csv")
df["Date first seen"] = pd.to_datetime(df["Date first seen"])
df = df[df["Date first seen"].dt.dayofweek.isin([1,5])]
df["pattern_instance"] = max_ids
df["pattern"] = used_pattern

In [8]:
df

,Date first seen,Proto,Src IP Addr,Dst IP Addr,Dst Pt,In Byte,Out Byte,pattern_instance,pattern
629878,2017-04-08 00:00:04.897,UDP,192.168.200.8,DNS,53.0,91.0,75.0,1,-1
629879,2017-04-08 00:00:05.305,UDP,192.168.100.4,DNS,53.0,131.0,74.0,1,-1
629880,2017-04-08 00:00:05.307,UDP,192.168.100.4,DNS,53.0,90.0,74.0,1,-1
629881,2017-04-08 00:00:05.429,UDP,192.168.100.4,DNS,53.0,131.0,74.0,1,-1
629882,2017-04-08 00:00:11.698,UDP,192.168.210.5,10031_250,1900.0,0.0,0.0,2,6
...,...,...,...,...,...,...,...,...,...
1089917,2017-04-11 23:59:27.532,TCP,192.168.200.8,EXT_SERVER,8000.0,800.0,646.0,144128,1338
1089918,2017-04-11 23:59:27.800,TCP,192.168.200.8,EXT_SERVER,8000.0,737.0,674.0,144129,1339
1089919,2017-04-11 23:59:30.112,UDP,192.168.200.8,10031_250,1900.0,0.0,0.0,144130,1325
1089920,2017-04-11 23:59:34.120,UDP,192.168.220.8,192.168.220.255,138.0,0.0,0.0,144131,1333


In [ ]:
df.to_csv("notebooks/data/cover_map_xp2.csv", index=False)

In [ ]:
pattern = 291
def highlight_sentiment(row):
    if row["pattern"] == pattern:
        return ['background-color: red'] * len(row)
    else:
        return ['background-color: black'] * len(row)

df.iloc[:5000].style.apply(highlight_sentiment, axis=1)

In [ ]:
df.iloc[:5000,-1].value_counts()

In [ ]:
df[df["pattern_instance"] == 291]

In [ ]:
u["Date first seen"] = pd.to_datetime(u["Date first seen"])

In [ ]:
usage_dict[291]

In [11]:
df.iloc[1135,:]

Date first seen     2017-04-08 01:20:20.839000
Proto                                    UDP  
Src IP Addr                      192.168.210.5
Dst IP Addr                          10031_250
Dst Pt                                  1900.0
In Byte                                    0.0
Out Byte                                   0.0
pattern_instance                           632
pattern                                      6
Name: 631013, dtype: object